In [1]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe

import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import java.util.UUID.randomUUID

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"
val ACCOUNT_MATCHERS_CSV_FILE = "../../data/provision/account-matchers.csv"
val FUND_MATCHERS_CSV_FILE = "../../data/provision/fund-matchers.csv"

// TODO(Johann) create a user
val userId = randomUUID()

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

In [2]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(userId).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
        .map { row -> CreateAccountTO(AccountName(row.getValue<String>("account")), Currency(row.getValue<String>("currency"))) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(userId, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=8bb01e3f-9464-40b2-9b85-cd236eed8377, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=e0c28f7f-41a1-4878-8900-7fa61f89ce1a, name=ING RON, unit=Currency(value=RON))
AccountTO(id=10ba857d-207b-4037-93e3-ce6a9b88072d, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=f263d4c6-557c-45d4-99c5-e12001e8b6c1, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=6d117623-e537-46b3-af30-4f9d8cbb33e5, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=c540438e-954f-4d59-a65c-f4c1dce38f12, name=Other, unit=Currency(value=RON))

In [3]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    // TODO(Johann) fix npe on get existing funds without accounts
    val existingFunds = fundSdk.listFunds(userId).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
        .map { row -> CreateFundTO(FundName(row.getValue<String>("fund"))) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(userId, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=1eead989-aa2d-40ec-96e2-fa2c38d452ac, name=Expenses)
FundTO(id=fe1606d9-a9e8-49cb-8b13-cd9687e85456, name=Work Income)
FundTO(id=72373246-cef7-475e-aa18-2df8d05e52d0, name=Savings)

In [4]:
import ro.jf.funds.importer.api.model.AccountMatcherTO

val accountMatchers = runBlocking {
    DataFrame.readCSV(ACCOUNT_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row -> AccountMatcherTO(row.getValue<String>("import_account_name"), AccountName(row.getValue<String>("account_name"))) }
}
accountMatchers.map { it.toString() }.joinToString(separator = "\n")

AccountMatcherTO(importAccountName=ING old, accountName=ING RON)
AccountMatcherTO(importAccountName=ING Economy old, accountName=ING Economy)
AccountMatcherTO(importAccountName=Cash RON, accountName=Cash RON)
AccountMatcherTO(importAccountName=Food Coupons, accountName=Food Coupons)
AccountMatcherTO(importAccountName=Revolut - Manual, accountName=Revolut RON)
AccountMatcherTO(importAccountName=Other, accountName=Other)

In [11]:
import ro.jf.funds.importer.api.model.*

val fundMatchers = runBlocking {
    DataFrame.readCSV(FUND_MATCHERS_CSV_FILE, delimiter = ';')
        .map { row ->
            when (row.getValue<String>("type")) {
                "by_account" -> FundMatcherTO.ByAccount(
                    importAccountName = row.getValue<String>("import_account_name"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )
                
                "by_label" -> FundMatcherTO.ByLabel(
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label" -> FundMatcherTO.ByAccountLabel(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                "by_account_label_with_transfer" -> FundMatcherTO.ByAccountLabelWithTransfer(
                    importAccountName = row.getValue<String>("import_account_name"),
                    importLabel = row.getValue<String>("import_label"),
                    initialFundName = FundName(row.getValue<String>("initial_fund_name")),
                    fundName = FundName(row.getValue<String>("fund_name"))
                )

                else -> error("fund matcher type not recognized ${row.getValue<String>("type")}")
            }
        }
}
fundMatchers.map { it.toString() }.joinToString(separator = "\n")

ByAccountLabelWithTransfer(importAccountName=ING Economy old, importLabel=Invest Profit Acc, initialFundName=Savings, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=ING old, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByAccountLabelWithTransfer(importAccountName=Food Coupons, importLabel=Work Income, initialFundName=Work Income, fundName=Expenses)
ByLabel(importLabel=Basic - Food, fundName=Expenses)
ByLabel(importLabel=C&T - Gas & Parking, fundName=Expenses)
ByLabel(importLabel=Development - Education, fundName=Expenses)
ByLabel(importLabel=F&V - Fun, fundName=Expenses)
ByLabel(importLabel=S&S - Objects, fundName=Expenses)
ByLabel(importLabel=Home - House, fundName=Expenses)
ByLabel(importLabel=Gifts, fundName=Expenses)
ByAccount(importAccountName=ING old, fundName=Expenses)
ByAccount(importAccountName=ING Economy old, fundName=Expenses)
ByAccount(importAccountName=Cash RON, fundName=Expenses)
ByAccount(importAccountName=Other, fundName=Exp

In [ ]:
// TODO(Johann) add exchange matchers

In [12]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

// TODO(Johann) load all files from the directory
val csvFiles = File("../../data/wallet/2019")
    .listFiles()?.toList() ?: error("no files found")
val importConfiguration = ImportConfigurationTO(
    fileType = ImportFileTypeTO.WALLET_CSV,
    accountMatchers = accountMatchers,
    fundMatchers = fundMatchers
)
runBlocking {
    var importTask = importSdk.import(userId, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(userId, importTask.taskId)
    }
    importTask
}


ro.jf.funds.commons.error.ApiException: ApiException: 400 - ErrorTO(title=Generic error, detail={"taskId":"ff47fda1-a266-4c0b-aac9-9fb392260989","status":"FAILED","reason":"Account name not matched: Euro"})